In [2]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
from tqdm import tqdm

# Tải các biến môi trường từ file .env
load_dotenv()

# Lấy khóa API từ biến môi trường
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("API key not found. Please set the GOOGLE_API_KEY in .env file.")

# Cung cấp khóa API cho mô hình Gemini
genai.configure(api_key=api_key)

# Thư mục gốc chứa các thư mục con
root_input_folder = "0.img-input"
output_folder = "2.txt-output"

# Tạo thư mục đầu ra nếu chưa tồn tại
os.makedirs(output_folder, exist_ok=True)

# Khởi tạo mô hình Gemini
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

# Lấy danh sách tất cả các thư mục con trong thư mục gốc
subfolders = [f.path for f in os.scandir(root_input_folder) if f.is_dir()]

# Lặp qua từng thư mục con
for input_folder in tqdm(subfolders, desc="Processing folders"):
    # Lấy tên thư mục con
    subfolder = os.path.basename(input_folder)
    
    # Lấy danh sách các file PNG trong thư mục con
    png_files = [f for f in os.listdir(input_folder) if f.endswith(".png")]
    
    # Xử lý từng file PNG trong thư mục con
    for filename in tqdm(png_files, desc=f"Processing {subfolder}"):
        image_path = os.path.join(input_folder, filename)
        
        # Upload file ảnh lên API và lấy thông tin file
        myfile = genai.upload_file(image_path)
        
        # Tạo prompt cho API
        prompt = "Trích xuất chính xác nội dung từ hình ảnh mà tôi cung cấp. Chú ý, chính xác từng kí tự, kể cả các ký tự đặc biệt. Xuống dòng giống hệt trong ảnh. Chỉ trả lời nội dung trong ảnh, không thêm thông tin khác."
        
        # Gọi API Gemini để trích xuất văn bản bằng URL của file ảnh
        response = model.generate_content([
            myfile,
            "\n\n",
            prompt
        ])
        
        # Lấy thông tin về số token từ phản hồi
        usage_metadata = response.usage_metadata
        print(f"Số token đầu vào cho {filename}: {usage_metadata.prompt_token_count}, "
              f"Số token đầu ra: {usage_metadata.candidates_token_count}, "
              f"Tổng số token: {usage_metadata.total_token_count}")

        # Tạo đường dẫn file đầu ra
        output_file_path = os.path.join(output_folder, subfolder, f"{os.path.splitext(filename)[0]}.txt")
        
        # Tạo thư mục con trong output nếu chưa tồn tại
        os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

        # Lưu văn bản trích xuất vào file txt
        with open(output_file_path, "w", encoding="utf-8") as output_file:
            output_file.write(response.text)

print("Text extraction completed and saved to the output folder.")

Processing folders:   0%|          | 0/1 [00:00<?, ?it/s]

Số token đầu vào cho page_5.png: 316, Số token đầu ra: 671, Tổng số token: 987


Số token đầu vào cho page_7.png: 316, Số token đầu ra: 1173, Tổng số token: 1489


Processing folders: 100%|██████████| 1/1 [00:47<00:00, 47.79s/it]

Số token đầu vào cho page_6.png: 316, Số token đầu ra: 1110, Tổng số token: 1426
Text extraction completed and saved to the output folder.
